# Final Project Submission for Capstone

* Student name: **Andrew Hotchkiss**
* Student pace: **Part time**
* Scheduled capstone review date/time: 
* Instructor name: **James Irving**
* Blog post URL: https://stones-1130.github.io/

## TABLE OF CONTENTS 

*Click to jump to matching Markdown Header.*<br><br>

<font size=3rem>
    
- **[Introduction](#INTRODUCTION)<br>**
- **[OBTAIN](#OBTAIN)**<br>
- **[SCRUB](#SCRUB)**<br>
- **[EXPLORE](#EXPLORE)**<br>
- **[MODEL](#MODEL)**<br>
- **[iNTERPRET](#iNTERPRET)**<br>
- **[Conclusions/Recommendations](#CONCLUSIONS-&-RECOMMENDATIONS)<br>**
</font>
___

# ABSTRACT

> briefly explains your problem, your methodology, and your findings, and business recommendations as a result of your findings. This section should be 1-2 paragraphs long

# INTRODUCTION

> README from the data set: 

"This dataset contains sensor data that was collected in the home of
a volunteer adult.  The resident in the home was a woman.  The woman's
children and grandchildren visited on a regular basis. The file contains
raw sensor events, no activities are annotated.

The sensor events are generated from motion sensors (these sensor IDs begin
with "M"), door closure sensors (these sensor IDs begin with "D"), and
temperature sensors (these sensor IDs begin with "T").
The layout of the sensors in the home is shown in the file aruba.jpg.

You may download and analyze this data free of charge.  Please do not
distribute the data without explicit permission from the CASAS research group.
Please reference the CASAS project in any publications resulting from using
this dataset.

All use of the data must cite the WSU CASAS smart home project.

D. Cook. Learning setting-generalized activity models for smart spaces.
IEEE Intelligent Systems, 2011."

> **Goal: Forecast nighttime bathroom use.**

# OBTAIN

#### IMPORT LIBRARIES AND PACKAGES

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm

#Project notebook settings
pd.set_option('display.max_columns', 0)

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-poster')

In [21]:
df = pd.read_csv('data_clean.csv', sep=' ', header=None, 
                  names=['Date', 'Time', 'Sensor', 'Reading', 'Event'])

df.shape

(1719558, 5)

In [22]:
df.reset_index(inplace=True)



In [23]:
df.columns = ['Date', 'Time', 'Sensor', 'Reading', 'Event', 'Event_2']

df[df['Time'].str.contains('\t')]


,Date,Time,Sensor,Reading,Event,Event_2


In [24]:
df

,Date,Time,Sensor,Reading,Event,Event_2
0,2010-11-04,00:03:50.209589,M003,ON,Sleeping,begin
1,2010-11-04,00:03:57.399391,M003,OFF,NaN,NaN
2,2010-11-04,00:15:08.984841,T002,21.5,NaN,NaN
3,2010-11-04,00:30:19.185547,T003,21,NaN,NaN
4,2010-11-04,00:30:19.385336,T004,21,NaN,NaN
...,...,...,...,...,...,...
1719553,2011-06-11,23:42:59.28507,T002,25.5,NaN,NaN
1719554,2011-06-11,23:48:02.888409,T001,23.5,NaN,NaN
1719555,2011-06-11,23:48:02.988798,T002,25,NaN,NaN
1719556,2011-06-11,23:53:06.4292,T002,25.5,NaN,NaN


In [25]:
df['Date_Time'] = pd.to_datetime(df['Date']+ ' ' + df['Time'])

df['Annotation'] = df['Event']+ ' ' + df['Event_2']

df

,Date,Time,Sensor,Reading,Event,Event_2,Date_Time,Annotation
0,2010-11-04,00:03:50.209589,M003,ON,Sleeping,begin,2010-11-04 00:03:50.209589,Sleeping begin
1,2010-11-04,00:03:57.399391,M003,OFF,NaN,NaN,2010-11-04 00:03:57.399391,NaN
2,2010-11-04,00:15:08.984841,T002,21.5,NaN,NaN,2010-11-04 00:15:08.984841,NaN
3,2010-11-04,00:30:19.185547,T003,21,NaN,NaN,2010-11-04 00:30:19.185547,NaN
4,2010-11-04,00:30:19.385336,T004,21,NaN,NaN,2010-11-04 00:30:19.385336,NaN
...,...,...,...,...,...,...,...,...
1719553,2011-06-11,23:42:59.28507,T002,25.5,NaN,NaN,2011-06-11 23:42:59.285070,NaN
1719554,2011-06-11,23:48:02.888409,T001,23.5,NaN,NaN,2011-06-11 23:48:02.888409,NaN
1719555,2011-06-11,23:48:02.988798,T002,25,NaN,NaN,2011-06-11 23:48:02.988798,NaN
1719556,2011-06-11,23:53:06.4292,T002,25.5,NaN,NaN,2011-06-11 23:53:06.429200,NaN


In [26]:
df.drop(['Date', 'Time', 'Event', 'Event_2'], axis=1, inplace=True)

df

,Sensor,Reading,Date_Time,Annotation
0,M003,ON,2010-11-04 00:03:50.209589,Sleeping begin
1,M003,OFF,2010-11-04 00:03:57.399391,NaN
2,T002,21.5,2010-11-04 00:15:08.984841,NaN
3,T003,21,2010-11-04 00:30:19.185547,NaN
4,T004,21,2010-11-04 00:30:19.385336,NaN
...,...,...,...,...
1719553,T002,25.5,2011-06-11 23:42:59.285070,NaN
1719554,T001,23.5,2011-06-11 23:48:02.888409,NaN
1719555,T002,25,2011-06-11 23:48:02.988798,NaN
1719556,T002,25.5,2011-06-11 23:53:06.429200,NaN


In [27]:
df1 = df.reindex(columns=['Date_Time', 'Sensor', 'Reading', 'Annotation'])

df1

,Date_Time,Sensor,Reading,Annotation
0,2010-11-04 00:03:50.209589,M003,ON,Sleeping begin
1,2010-11-04 00:03:57.399391,M003,OFF,NaN
2,2010-11-04 00:15:08.984841,T002,21.5,NaN
3,2010-11-04 00:30:19.185547,T003,21,NaN
4,2010-11-04 00:30:19.385336,T004,21,NaN
...,...,...,...,...
1719553,2011-06-11 23:42:59.285070,T002,25.5,NaN
1719554,2011-06-11 23:48:02.888409,T001,23.5,NaN
1719555,2011-06-11 23:48:02.988798,T002,25,NaN
1719556,2011-06-11 23:53:06.429200,T002,25.5,NaN


In [19]:
df1.set_index('Date_Time', inplace=True)

df1

,Sensor,Reading,Annotation
Date_Time,,,
2010-11-04 00:03:50.209589,M003,ON,Sleeping begin
2010-11-04 00:03:57.399391,M003,OFF,NaN
2010-11-04 00:15:08.984841,T002,21.5,NaN
2010-11-04 00:30:19.185547,T003,21,NaN
2010-11-04 00:30:19.385336,T004,21,NaN
...,...,...,...
2011-06-11 23:42:59.285070,T002,25.5,NaN
2011-06-11 23:48:02.888409,T001,23.5,NaN
2011-06-11 23:48:02.988798,T002,25,NaN


In [9]:
df1['Annotation'].value_counts()

Relax end                 2909
Relax begin               2884
Meal_Preparation end      1596
Meal_Preparation begin    1581
Leave_Home end             431
Leave_Home begin           431
Enter_Home end             431
Enter_Home begin           431
Sleeping end               401
Sleeping begin             395
Eating end                 255
Eating begin               251
Work end                   170
Work begin                 168
Bed_to_Toilet begin        155
Bed_to_Toilet end          154
Wash_Dishes end             64
Wash_Dishes begin           61
Housekeeping end            33
Housekeeping begin          33
Respirate end                6
Respirate begin              5
Name: Annotation, dtype: int64

In [ ]:
#ELIMINATE THE ROWS WE DON'T CARE ABOUT - SPECIFICALLY TEMPERATURE


# EXPLORE

In [10]:
df.shape

(1719558, 4)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1719558 entries, 0 to 1719557
Data columns (total 4 columns):
Sensor        object
Reading       object
Date_Time     datetime64[ns]
Annotation    object
dtypes: datetime64[ns](1), object(3)
memory usage: 52.5+ MB


In [12]:
print(df.dtypes)

Sensor                object
Reading               object
Date_Time     datetime64[ns]
Annotation            object
dtype: object


In [ ]:
df['Time'].value_counts()

In [ ]:
df['Sensor'].value_counts()

# SCRUB

# MODEL

# iNTERPRET

# CONCLUSIONS AND RECOMMENDATIONS

In [ ]:
ROI_df2

> **Looking at the dataframe above, there are only three zip codes (83706, 83713, 83714) that have a low probability of a negative ROI on a two year time horizon.**

> **83704 and 83716 round out our 5 least risky investments, although they do have show a potential negative ROI.**

## FUTURE WORK

> - **Dig deeper into auto_arima to see if there are better options for hyper parameter tuning to improve our models' fit.**

> - **Get more recent data (the current data set only includes dates up to April 2018) to be able to forecast more accurately into the future.**

> - **Research Zillow "Zestimate" calculation for home sale price forecasting.**